# TF-IDF

We'll rely on term frequency times inverted document frequency to measure meaningful similarity between documents. Let's start by generating a matrix for the separate constituent parts of _Stjórn_.

## TODO

Go back to the generate-scripts and filter out accented vowels as well as eth

In [ ]:
import os,glob,json
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
#from unidecode import unidecode

In [ ]:
titles = ['prologue', 'introduction', 'gn', 'ex', 'lv', 'nm', 'dt', 'ios', 'idc', 'rt', '1sm', '2sm', '3rg', '4rg']
tokens = []
for title in titles:
    with open(f"nlp/{title}.txt") as raw:
        document = raw.read().replace('\n', ' ')
        tokens.extend(document.split())

work_indices = {
    'stjorn1': (650,124417),
    'stjorn2': (124417,147678),
    'stjorn3': (147678,156943,160719),
    'stjorn4': (156943,160719)
}

stjorn = dict()
for _work, _range in work_indices.items():
    if len(_range) == 2:
        stjorn[_work] = ' '.join(tokens[_range[0]:_range[1]])
    else:
        stjorn[_work] = ' '.join(tokens[_range[0]:_range[1]] + tokens[_range[2]:])

menota = dict()
for text in glob.glob('../menota/dipl/*txt'):
    ref = os.path.basename(text).replace('.txt', '')
    with open(text) as doc:
        menota[ref] = doc.read().replace('\n', '')

In [8]:
vectorizer = TfidfVectorizer(min_df=1)
model = vectorizer.fit_transform(stjorn.values())
df = pd.DataFrame(cosine_similarity(model), stjorn.keys(), stjorn.keys())
df

stjorn1   stjorn2   stjorn3   stjorn4
stjorn1  1.000000  0.811713  0.508459  0.842683
stjorn2  0.811713  1.000000  0.394915  0.833537
stjorn3  0.508459  0.394915  1.000000  0.469750
stjorn4  0.842683  0.833537  0.469750  1.000000

This tells us _Stjórn III_ is the most distinct of the three. Remarkably, compared to _Stjórn IV_, which covers some of the same ground, it is the least similar constituent text. Perhaps further analysis can tell us how.

First let's add _Konungs skuggsjá_ from Menota, as well as Unger's own edition of the _Norwegian Homily Book_. Fingers crossed that we have got the normalization standard of the former to approach Unger's methods reasonably well.

In [9]:
nhb_titles = ['alcuin', 'hom', 'olafr', 'visio', 'paternoster', 'anhang1'] # this is the sequence matched in Menota
nhb = ''
for title in nhb_titles:
    filepath = f'../nhb/nlp/{title}.txt'
    with open(filepath) as doc:
        nhb = nhb + doc.read().replace('\n', '')
stjorn_plus = []
for v in stjorn.values():
    stjorn_plus.append(v)
stjorn_plus.extend([menota['nks235g_konungs_skuggsja'], nhb])
model = vectorizer.fit_transform(stjorn_plus)
df = pd.DataFrame(cosine_similarity(model), list(stjorn.keys()) + ['ks', 'nhb'], list(stjorn.keys()) + ['ks', 'nhb'])
df

stjorn1   stjorn2   stjorn3   stjorn4        ks       nhb
stjorn1  1.000000  0.786055  0.523931  0.812032  0.242538  0.385928
stjorn2  0.786055  1.000000  0.402580  0.797535  0.228309  0.329922
stjorn3  0.523931  0.402580  1.000000  0.474961  0.696934  0.827073
stjorn4  0.812032  0.797535  0.474961  1.000000  0.244245  0.385029
ks       0.242538  0.228309  0.696934  0.244245  1.000000  0.710246
nhb      0.385928  0.329922  0.827073  0.385029  0.710246  1.000000

Next, let's model all of Menota along with Stjórn. Perhaps we'll leave Unger's _Homily Book_ in alongside the Menota edition, just for comparison's sake.

In [ ]:
corpus = []
titles = []
for k,v in stjorn.items():
    titles.append(k)
    corpus.append(v)
titles.append('nhb')
corpus.append(nhb)
for k,v in menota.items():
    titles.append(k)
    corpus.append(v)
model = vectorizer.fit_transform(corpus)
df = pd.DataFrame(cosine_similarity(model), titles, titles)
df

stjorn1   stjorn2   stjorn3  \
stjorn1                                    1.000000  0.748352  0.463369   
stjorn2                                    0.748352  1.000000  0.367732   
stjorn3                                    0.463369  0.367732  1.000000   
stjorn4                                    0.716519  0.690449  0.422768   
nhb                                        0.321304  0.278872  0.804556   
am132_egils_saga                           0.729321  0.683951  0.423967   
am162btheta_njals_saga                     0.285188  0.228120  0.735041   
holmPerg30_langslog                        0.450608  0.471444  0.264177   
am1056IX_konungs_skuggsja_fragment         0.460412  0.422574  0.232953   
am78_kristinrettir                         0.185049  0.194668  0.575662   
am63_heimskringla3                         0.273708  0.234338  0.735079   
dg4-7_strengleikar                         0.264076  0.212417  0.819889   
am132_droplaugasona_saga                   0.665099  0.640855  0.337100   
am132_kormaks_saga                         0.560652  0.535829  0.294350   
nraNorrFragm53_haralds_saga_hardrada       0.468344  0.436430  0.269418   
am132_finnboga_saga                        0.660071  0.605030  0.354904   
am132_fostbraedra_saga                     0.695973  0.652222  0.373207   
lbsFragm82_olafs_saga_helga                0.225799  0.192477  0.561057   
am619_norwegian_homily_book                0.672364  0.646777  0.414823   
am56_landslog                              0.217732  0.222568  0.580016   
holmPerg17_thomass_saga                    0.624829  0.443717  0.579391   
am383I_thorlaks_saga                       0.215162  0.174099  0.448595   
holmPerg4_thidreks_saga                    0.288862  0.269812  0.828474   
am132_njals_saga                           0.744594  0.706287  0.415510   
am36_heimskringla2                         0.309130  0.265313  0.758734   
am544_voluspa                              0.405335  0.382553  0.189421   
am162bkappa_njals_saga                     0.262443  0.226391  0.664056   
am305_landslog                             0.197563  0.202864  0.544162   
am132_olkofra_thattr                       0.572001  0.558609  0.392927   
nraNorrFragm54_sverris_saga                0.352587  0.328450  0.790717   
nraNorrFragm55B_hakonar_saga               0.190760  0.193694  0.550567   
gks2365_voluspa                            0.128820  0.117150  0.413245   
am243balpha_konungs_skuggsja               0.275549  0.237192  0.686689   
nraNorrFragm51_fagrskinna                  0.119370  0.118224  0.491174   
am132_viga-glums_saga                      0.726521  0.685721  0.401977   
am279a_gragas                              0.226857  0.204265  0.450052   
am677_gregory                              0.531046  0.508069  0.340567   
am132_laxdoela_saga                        0.721918  0.680016  0.416909   
am302_landslog                             0.193953  0.200113  0.534692   
am178_thidreks_saga                        0.165463  0.183253  0.149867   
am132_bandamanna_saga                      0.675692  0.647547  0.398944   
am655_laeknisbok                           0.147910  0.139526  0.543393   
am519a_alexanders_saga                     0.525073  0.535123  0.331151   
holmPerg34_landslog                        0.624790  0.653535  0.370103   
am162balpha_njals_saga                     0.613095  0.584767  0.345557   
am113_islendingabok                        0.171652  0.135629  0.448123   
nraNorrFragm7_landslog                     0.239980  0.240224  0.573009   
nks235g_konungs_skuggsja                   0.190201  0.186604  0.660723   
am132_hallfredar_saga                      0.638843  0.615179  0.358496   
am35_heimskringla1                         0.263255  0.228013  0.747495   
am242_codex_wormianus                      0.772092  0.700719  0.434986   
dg8II_olafs_saga                           0.240468  0.225097  0.755522   
am28_codex_runicus                         0.308482  0.278706  0.1189

That's not a good sign, two editions of the _Norwegian Homily Book_ doing no better than `0.54` similarity... If I run `unidecode` (which strips out accented vowels, among other things), it only gets up to `0.55`.